In [39]:
%run imports_and_functions.ipynb

### Cleaning and Exporting Final Las Vegas Dataset

In [2]:
df = pd.read_csv('LasVegasBusinesses.csv', compression='gzip', sep=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28865 entries, 0 to 28864
Data columns (total 12 columns):
address         27073 non-null object
attributes      24706 non-null object
categories      28785 non-null object
city            28865 non-null object
latitude        28864 non-null float64
longitude       28864 non-null float64
name            28865 non-null object
neighborhood    23607 non-null object
postal_code     28765 non-null float64
review_count    28865 non-null int64
stars           28865 non-null float64
state           28865 non-null object
dtypes: float64(4), int64(1), object(7)
memory usage: 2.6+ MB


In [10]:
df['postal_code'].replace(np.nan, 0, inplace=True)

In [11]:
df[df['postal_code'].isnull()]

,address,attributes,categories,city,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state


In [15]:
df.loc[:,'postal_code'] = df['postal_code'].astype(int, copy=False)

In [96]:
df.tail()

,address,attributes,categories,city,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,price
28860,4655 Quality Ct,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Kitchen & Bath, Building Supplies, Home Servic...",Las Vegas,36.105866,-115.206215,Impressions Tile & Marble,NaN,89103,6,4.5,NV,0
28861,"3863 S Valley View Blvd, Ste 31",{'BusinessAcceptsCreditCards': 'True'},"Automotive, Body Shops, Auto Repair, Oil Chang...",Las Vegas,36.120743,-115.192017,Vegas Auto Repair,NaN,89103,9,4.5,NV,0
28862,4322 W Flamingo Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Beauty & Spas, Nail Salons, Active Life, Gyms,...",Las Vegas,36.114304,-115.193871,Recharge Salon & Fitness,NaN,89103,20,4.5,NV,3
28863,6166 W Flamingo Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Beauty & Spas, Nail Salons, Hair Removal, Waxi...",Las Vegas,36.115367,-115.226848,Aloha Nails,NaN,89103,76,3.5,NV,2
28864,"3900 Paradise Rd, Ste R","{'BikeParking': 'True', 'BusinessAcceptsCredit...","Makeup Artists, Men's Clothing, Swimwear, Shop...",Las Vegas,36.118697,-115.154270,Tito Jimy Salon and Boutique,Eastside,89169,8,4.0,NV,2


In [35]:
'active life' in df.tail().iloc[2,2].lower()

True

In [23]:
df[~(df['postal_code'].isin(vegas_zipcodes))]['postal_code'].value_counts()

0         100
88901      63
89032      45
89030      45
89031      32
89052      23
89014      20
89084      17
89074      16
89081      11
89012       7
89011       5
89019       5
90210       2
89085       2
89015       1
85705       1
86023       1
88113       1
78409       1
88147       1
89181       1
89184       1
89053       1
6280        1
87701       1
92328       1
71863       1
891118      1
89018       1
89046       1
88905       1
93013       1
98145       1
68451       1
94066       1
89039       1
89003       1
89004       1
90720       1
89037       1
90001       1
Name: postal_code, dtype: int64

In [19]:
vegas_zipcodes = [
89044,89054,89101,89102,89103,89104,89105,89106,89107,89108,89109,89110,89111,89112,89113,89114,
89115,89116,89117,89118,89119,89120,89121,89122,89123,89124,89125,89126,89127,89128,89129,89130,
89131,89132,89133,89134,89135,89136,89137,89138,89139,89140,89141,89142,89143,89144,89145,89146,
89147,89148,89149,89150,89151,89152,89153,89154,89155,89156,89157,89158,89159,89160,89161,89162,
89163,89164,89165,89166,89169,89170,89173,89177,89178,89179,89180,89183,89185,89191,89193,89195,89199]
# From https://data.mongabay.com/igapo/zip_codes/NV.htm#i5DwGpyATaUgxp84.99

In [27]:
df.loc[~(df['postal_code'].isin(vegas_zipcodes)),'postal_code'] = 0

In [93]:
def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0

In [94]:
getPriceRange(df.loc[4,'attributes'])

1

In [95]:
df['price'] = df['attributes'].map(getPriceRange)
df['price'].value_counts()

0    14236
2     8041
1     4798
3     1317
4      473
Name: price, dtype: int64

In [20]:
def get_main_category(catstring) :
    string = ''
    try :
        for cat in main_categories :
            if cat in catstring : string = string + cat + ', '
    except : pass
    return string[0:(len(string)-2)]

In [21]:
get_main_category(df['categories'][0])

'Shopping'

In [22]:
df['main_category'] = df['categories'].map(get_main_category)
df['main_category'].value_counts()

Restaurants                                                                                                3115
Shopping                                                                                                   2467
Home Services                                                                                              2181
Beauty & Spas                                                                                              2060
Health & Medical                                                                                           2016
Food, Restaurants                                                                                          1884
Automotive                                                                                                 1484
Food                                                                                                       1176
Local Services                                                                                          

In [23]:
df.tail()

,address,attributes,categories,city,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,price,main_category
28860,4655 Quality Ct,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Kitchen & Bath, Building Supplies, Home Servic...",Las Vegas,36.105866,-115.206215,Impressions Tile & Marble,NaN,89103,6,4.5,NV,0,"Home Services, Shopping"
28861,"3863 S Valley View Blvd, Ste 31",{'BusinessAcceptsCreditCards': 'True'},"Automotive, Body Shops, Auto Repair, Oil Chang...",Las Vegas,36.120743,-115.192017,Vegas Auto Repair,NaN,89103,9,4.5,NV,0,Automotive
28862,4322 W Flamingo Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Beauty & Spas, Nail Salons, Active Life, Gyms,...",Las Vegas,36.114304,-115.193871,Recharge Salon & Fitness,NaN,89103,20,4.5,NV,3,"Active Life, Beauty & Spas"
28863,6166 W Flamingo Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Beauty & Spas, Nail Salons, Hair Removal, Waxi...",Las Vegas,36.115367,-115.226848,Aloha Nails,NaN,89103,76,3.5,NV,2,Beauty & Spas
28864,"3900 Paradise Rd, Ste R","{'BikeParking': 'True', 'BusinessAcceptsCredit...","Makeup Artists, Men's Clothing, Swimwear, Shop...",Las Vegas,36.118697,-115.154269,Tito Jimy Salon and Boutique,Eastside,89169,8,4.0,NV,2,"Beauty & Spas, Shopping"


In [24]:
df.to_csv('LasVegas.csv',sep=';',header=True,index=False,compression='gzip')

### Business Vectorize Functions for a Location 

In [ ]:
df = pd.read_csv('LasVegas.csv',sep=';',compression='gzip')

In [26]:
testlat = 36.120743
testlong = -115.192017 # same coords as autorepair shop in tail of las vegas dataset

In [31]:
def calcDistances(lat, long, citydf) :
    distances = pd.Series(index=citydf.index)
    for index in citydf.index :
        distances[index] = calcDist(lat, long, citydf.loc[index,'latitude'], citydf.loc[index,'longitude'])
    return distances

In [40]:
results = calcDistances(testlat, testlong, df)
results[results < search_radius]

124      756.578301
208      765.864667
255      973.197697
414      460.796078
569      738.852141
608      558.352126
702      515.163792
748      470.980523
978      942.223872
1054     780.714841
1085     757.099413
1316     625.965896
1405     682.566558
1419     923.452452
1616     727.021330
1799     807.826330
1800     112.244559
1837     774.976823
1998     500.465278
2041     555.086623
2052     842.593902
2189     938.324328
2241      81.875111
2311     111.693278
2374     738.819004
2400     549.937323
2444     646.253775
2445     808.289048
2602     845.699750
3009     610.566873
3177     269.861606
3194     594.179987
3251     652.673844
3561     793.966080
3566     780.592592
3694     593.037600
3896     734.244402
3908     702.917976
4026     492.257434
4030     475.775473
4117     799.044674
4251     394.917450
4340     916.518562
4467     626.201874
4591     684.898624
4592     671.926328
4607     474.757591
4716     551.012449
4767     947.815840
4900     895.142118


In [41]:
results[results < search_radius].shape

(555,)

In [43]:
df.loc[results[results < search_radius].index,'price'].value_counts()

0    225
2    185
1     98
3     39
4      8
Name: price, dtype: int64

In [46]:
df['postal_code'].value_counts()  # 74 unique zip codes plus 0 for mislabeled or missing zipcodes

89109    3116
89119    1808
89102    1618
89103    1483
89117    1376
89118    1340
89123    1206
89146    1182
89101    1182
89147    1116
89104     981
89121     880
89128     855
89148     727
89130     647
89113     636
89139     614
89120     603
89108     561
89145     536
89135     524
89169     512
89149     510
89106     484
89129     440
89183     423
0         420
89107     415
89131     340
89115     340
89110     315
89134     259
89122     230
89141     180
89144     169
89178     140
89142     101
89136     101
89156      64
89143      56
89158      55
89166      50
89138      37
89161      27
89154      25
89124      23
89162      21
89044      20
89111      18
89133      10
89191       9
89193       9
89179       9
89105       7
89155       5
89180       5
89140       4
89112       4
89199       4
89157       3
89165       3
89152       3
89153       3
89137       3
89151       3
89160       2
89114       2
89054       2
89173       2
89116       2
89126       1
89125 

In [53]:
Ydata = pd.read_csv('lasvegaszipcodes.csv')
Ydata

,#,Zip Code,Location,City,Population,Avg. Income/H/hold
0,1,89141,"35.982470, -115.205227","Las Vegas, Nevada",262,"$108,579.00"
1,2,89148,"36.063771, -115.315877","Las Vegas, Nevada",913,"$92,707.00"
2,3,89139,"36.032308, -115.208632","Las Vegas, Nevada","2,297","$74,706.00"
3,4,89149,"36.279319, -115.300319","Las Vegas, Nevada","11,312","$74,482.00"
4,5,89144,"36.171202, -115.345319","Las Vegas, Nevada","11,772","$70,136.00"
5,6,89143,"36.659049, -115.185865","Las Vegas, Nevada","2,408","$67,411.00"
6,7,89129,"36.235825, -115.298399","Las Vegas, Nevada","32,441","$63,373.00"
7,8,89135,"36.119441, -115.323855","Las Vegas, Nevada","3,539","$62,629.00"
8,9,89131,"36.294631, -115.235476","Las Vegas, Nevada","10,968","$61,779.00"
9,10,89113,"36.060558, -115.263553","Las Vegas, Nevada","8,276","$61,057.00"


In [51]:
np.array(set(df['postal_code']).difference(set(Ydata['Zip Code'])))

array({0, 89105, 89111, 89112, 89114, 89116, 89125, 89126, 89132, 89133, 89136, 89137, 89138, 89140, 89150, 89151, 89152, 89153, 89154, 89155, 89157, 89158, 89160, 89161, 89162, 89165, 89166, 89169, 89044, 89173, 89178, 89179, 89180, 89054, 89183, 89191, 89193, 89195, 89199},
      dtype=object)

In [42]:
df.tail()

,address,attributes,categories,city,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,price,main_category
28860,4655 Quality Ct,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Kitchen & Bath, Building Supplies, Home Servic...",Las Vegas,36.105866,-115.206215,Impressions Tile & Marble,NaN,89103,6,4.5,NV,0,"Home Services, Shopping"
28861,"3863 S Valley View Blvd, Ste 31",{'BusinessAcceptsCreditCards': 'True'},"Automotive, Body Shops, Auto Repair, Oil Chang...",Las Vegas,36.120743,-115.192017,Vegas Auto Repair,NaN,89103,9,4.5,NV,0,Automotive
28862,4322 W Flamingo Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Beauty & Spas, Nail Salons, Active Life, Gyms,...",Las Vegas,36.114304,-115.193871,Recharge Salon & Fitness,NaN,89103,20,4.5,NV,3,"Active Life, Beauty & Spas"
28863,6166 W Flamingo Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Beauty & Spas, Nail Salons, Hair Removal, Waxi...",Las Vegas,36.115367,-115.226848,Aloha Nails,NaN,89103,76,3.5,NV,2,Beauty & Spas
28864,"3900 Paradise Rd, Ste R","{'BikeParking': 'True', 'BusinessAcceptsCredit...","Makeup Artists, Men's Clothing, Swimwear, Shop...",Las Vegas,36.118697,-115.154269,Tito Jimy Salon and Boutique,Eastside,89169,8,4.0,NV,2,"Beauty & Spas, Shopping"


### Writing csv files for US Cities from Kaggle's Yelp Business Dataset

In [2]:
test = pd.read_json('yelp_academic_dataset_business.json', lines=True)
test.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",MontrÃ©al,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,None,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB


In [6]:
test.iloc[0,3]

'Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel'

In [5]:
test.drop('hours', axis=1, inplace=True)

In [7]:
test.drop('is_open', axis=1, inplace=True)

In [8]:
test.drop('business_id', axis=1, inplace=True)

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 12 columns):
address         188593 non-null object
attributes      162807 non-null object
categories      188052 non-null object
city            188593 non-null object
latitude        188587 non-null float64
longitude       188587 non-null float64
name            188593 non-null object
neighborhood    188593 non-null object
postal_code     188593 non-null object
review_count    188593 non-null int64
stars           188593 non-null float64
state           188593 non-null object
dtypes: float64(3), int64(1), object(8)
memory usage: 17.3+ MB


In [16]:
test[test['city']=='Las Vegas'].to_csv('LasVegasBusinesses.csv',
                                      sep=';',
                                      header=True,
                                      index=False,
                                      compression='gzip')

In [17]:
test[test['city']=='Phoenix'].to_csv('PhoenixBusinesses.csv',
                                      sep=';',
                                      header=True,
                                      index=False,
                                      compression='gzip')

In [18]:
test[test['city']=='Charlotte'].to_csv('CharlotteBusinesses.csv',
                                      sep=';',
                                      header=True,
                                      index=False,
                                      compression='gzip')

In [19]:
test[test['city']=='Pittsburgh'].to_csv('PittsburghBusinesses.csv',
                                      sep=';',
                                      header=True,
                                      index=False,
                                      compression='gzip')

### Business Density Plot by Jagan Gupta from Kaggle

In [ ]:
# Code from https://www.kaggle.com/jagangupta for Kaggle's yelp challenge in
# https://www.kaggle.com/jagangupta/what-s-in-a-review-yelp-ratings-eda/notebook


import imageio
import folium
import folium.plugins as plugins
from mpl_toolkits.basemap import Basemap

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,7))

#a random point inside vegas
lat = 36.207430
lon = -115.268460
#some adjustments to get the right pic
lon_min, lon_max = lon-0.3,lon+0.5
lat_min, lat_max = lat-0.4,lat+0.5
#subset for vegas
ratings_data_vegas=rating_data[(rating_data["longitude"]>lon_min) &\
                    (rating_data["longitude"]<lon_max) &\
                    (rating_data["latitude"]>lat_min) &\
                    (rating_data["latitude"]<lat_max)]

#Facet scatter plot
ratings_data_vegas.plot(kind='scatter', x='longitude', y='latitude',
                color='yellow', 
                s=.02, alpha=.6, subplots=True, ax=ax1)
ax1.set_title("Las Vegas")
ax1.set_facecolor('black')

#a random point inside pheonix
lat = 33.435463
lon = -112.006989
#some adjustments to get the right pic
lon_min, lon_max = lon-0.3,lon+0.5
lat_min, lat_max = lat-0.4,lat+0.5
#subset for pheonix
ratings_data_pheonix=rating_data[(rating_data["longitude"]>lon_min) &\
                    (rating_data["longitude"]<lon_max) &\
                    (rating_data["latitude"]>lat_min) &\
                    (rating_data["latitude"]<lat_max)]
#plot pheonix
ratings_data_pheonix.plot(kind='scatter', x='longitude', y='latitude',
                color='yellow', 
                s=.02, alpha=.6, subplots=True, ax=ax2)
ax2.set_title("Pheonix")
ax2.set_facecolor('black')
f.show()

In [ ]:
data=[]
#rearranging data to suit the format needed for folium
stars_list=list(rating_data['stars'].unique())
for star in stars_list:
    subset=ratings_data_vegas[ratings_data_vegas['stars']==star]
    data.append(subset[['latitude','longitude']].values.tolist())
#initialize at vegas
lat = 36.127430
lon = -115.138460
zoom_start=11
print("                     Vegas Review heatmap Animation ")

# basic map
m = folium.Map(location=[lat, lon], tiles="OpenStreetMap", zoom_start=zoom_start)
#inprovising the Heatmapwith time plugin to show variations across star ratings 
hm = plugins.HeatMapWithTime(data,max_opacity=0.3,auto_play=True,display_index=True,radius=7)
hm.add_to(m)
m